In [1]:
import requests
from bs4 import BeautifulSoup
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
# URL del top 100 de libros
url = 'https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page'

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar si la solicitud fue exitosa 
if response.status_code == 200:
    # Analizar el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Buscar el elemento <div> con la clase "faq-answers" y el id "faq2023"
    target_div = soup.find('div', {'class': 'faq-answers', 'id': 'faq2023'})

    if target_div:
        # Buscar los enlaces <a> dentro de target_div que tienen title="Yellow Taxi Trip Records"
        yellow_taxi_links = target_div.find_all('a', title='Yellow Taxi Trip Records')

        if yellow_taxi_links:
            print("Enlaces con el título 'Yellow Taxi Trip Records':")
            links = []
            for link in yellow_taxi_links:
                links.append(link.get('href'))
                print(link.get('href'))  # Imprime el enlace URL
        else:
            print("No se encontraron enlaces con el título 'Yellow Taxi Trip Records' dentro de 'faq-answers'.")

    else:
        print("No se encontró el elemento con la clase 'faq-answers' y el id 'faq2023'.")

else:
    print("La solicitud no fue exitosa. Código de estado:", response.status_code)


Enlaces con el título 'Yellow Taxi Trip Records':
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-04.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet 
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-06.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-07.parquet 


In [3]:
def enviar_correo(asunto, cuerpo):
    remitente = 'aymara_falcon@mi.unc.edu.ar'
    destinatario = 'aymaratfferreyra@gmail.com'
    contraseña = 'MVAdan2020'

    mensaje = MIMEMultipart()
    mensaje['From'] = remitente
    mensaje['To'] = destinatario
    mensaje['Subject'] = asunto

    mensaje.attach(MIMEText(cuerpo, 'plain'))

    servidor_smtp = smtplib.SMTP('smtp.gmail.com', 587)
    servidor_smtp.starttls()
    servidor_smtp.login(remitente, contraseña)
    servidor_smtp.sendmail(remitente, destinatario, mensaje.as_string())
    servidor_smtp.quit()

In [4]:
# Carpeta de destino para guardar los archivos descargados
carpeta_destino = "archivos_yellow_taxi/"

# Verificar si la carpeta de destino existe, si no, créala
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

# Obtener la lista de archivos en la carpeta de destino
archivos_locales = os.listdir(carpeta_destino)

# Realiza la descarga de los archivos que no están en la lista de archivos locales
for link in links:
    nombre_archivo = link.split("/")[-1]  # Obtiene el nombre del archivo desde la URL
    archivo_destino = os.path.join(carpeta_destino, nombre_archivo)

    if nombre_archivo not in archivos_locales:
        response = requests.get(link)

        if response.status_code == 200:
            with open(archivo_destino, 'wb') as archivo:
                archivo.write(response.content)
            print(f"Archivo descargado: {nombre_archivo}")
        else:
            # Obtiene el motivo del error de la respuesta HTTP
            motivo_error = response.text
            # Muestra un mensaje indicando que el archivo no se pudo descargar y el motivo del error
            print(f"No se pudo descargar el archivo: {nombre_archivo}. Motivo del error: {motivo_error}")
            # Envía un correo electrónico notificando sobre el problema
            enviar_correo("Error de descarga", f"No se pudo descargar el archivo: {nombre_archivo}. Motivo del error: {motivo_error}")
    else:
        print(f"El archivo {nombre_archivo} ya existe localmente.")

El archivo yellow_tripdata_2023-01.parquet ya existe localmente.
El archivo yellow_tripdata_2023-02.parquet ya existe localmente.
El archivo yellow_tripdata_2023-03.parquet ya existe localmente.
El archivo yellow_tripdata_2023-04.parquet ya existe localmente.
No se pudo descargar el archivo: yellow_tripdata_2023-05.parquet . Motivo del error: <?xml version="1.0" encoding="UTF-8"?>
<Error><Code>AccessDenied</Code><Message>Access Denied</Message><RequestId>FYNRCB0VFS919EB3</RequestId><HostId>/vs4poHuMJFhh3FTDSwnfqrcE5gL68toElDVsZVICOQL3DeoozmlNU2oiyTQY8C38/hbQhUkZIo=</HostId></Error>


SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials fc10-20020a05690c314a00b005a7bf9749c8sm1477037ywb.4 - gsmtp')